In [ ]:
import pandas as pd
import pdfplumber

# 1. Read and group by ISIN in Excel
excel_file = 'your_isin_file.xlsx'
df_excel = pd.read_excel(excel_file)

# Group by ISIN and sum the amount
df_grouped = df_excel.groupby('ISIN')['Amount'].sum().reset_index()

# 2. Extract amount from PDF (assuming a table structure)
pdf_file = 'your_client_pdf.pdf'
with pdfplumber.open(pdf_file) as pdf:
    for page in pdf.pages:
        table = page.extract_table()
        # Assuming ISIN and amount are in the first and second columns respectively
        pdf_data = pd.DataFrame(table[1:], columns=table[0])  # convert to DataFrame for easier manipulation
        # Find the row where ISIN matches
        pdf_isin_row = pdf_data[pdf_data['Fund ISIN'] == 'Your ISIN']

# Get the relevant amount from PDF row
pdf_isin_amount = pdf_isin_row['Amount'].astype(float).sum()

# 3. Create comparison DataFrame
comparison_df = pd.DataFrame({
    'ISIN': df_grouped['ISIN'],
    'Excel Amount': df_grouped['Amount'],
    'PDF Amount': pdf_isin_amount
})

# Calculate the difference and percentage difference
comparison_df['Difference'] = comparison_df['Excel Amount'] - comparison_df['PDF Amount']
comparison_df['% Difference'] = (comparison_df['Difference'] / comparison_df['Excel Amount']) * 100

print(comparison_df)


In [ ]:
import pandas as pd
import re

# Load the Excel file
excel_file = 'your_isin_file_with_multiple_sheets.xlsx'
all_sheets = pd.read_excel(excel_file, sheet_name=None)  # Load all sheets into a dictionary

# Initialize lists to store the extracted data
isins = []
trader_names_list = []
usd_diffs = []
percent_diffs = []
sheet_names = []

# Function to clean sheet names by removing numbers in brackets
def clean_sheet_name(sheet_name):
    return re.sub(r"\s*\(.*?\)", "", sheet_name)

# Process each sheet in the Excel file
for sheet_name, df in all_sheets.items():
    
    # Only process if the sheet has entries (non-empty DataFrame)
    if len(df) > 0:
        # Variables to hold current ISIN and trader names for each sheet
        current_isin = None
        trader_names = []
        
        # Loop through the DataFrame to group ISINs and extract necessary data
        for idx, row in df.iterrows():
            isin = row.get('ISIN')
            trader = row.get('Trader name')
            
            # If ISIN is not NaN, this is part of the ISIN group
            if pd.notna(isin):
                if current_isin is None:  # Start a new ISIN group
                    current_isin = isin
                trader_names.append(trader)
            
            # If ISIN and Trader are NaN, indicating the blank row with difference data
            elif pd.isna(isin) and pd.isna(trader):
                if current_isin:  # Finalize the group for the current ISIN
                    # Append the data for the ISIN group
                    isins.append(current_isin)
                    trader_names_list.append(', '.join(trader_names))  # Concatenate trader names
                    usd_diffs.append(row['Diff'])  # Extract USD Difference
                    percent_diffs.append(row['%diff'])  # Extract % Difference
                    sheet_names.append(clean_sheet_name(sheet_name))  # Add cleaned sheet name
                    
                    # Reset for the next ISIN group
                    current_isin = None
                    trader_names = []

# Create a new DataFrame with the extracted data, including the sheet name
result_df = pd.DataFrame({
    'ISIN': isins,
    'Trader Names': trader_names_list,
    'USD Difference': usd_diffs,
    '% Difference': percent_diffs,
    'Sheet Name': sheet_names
})

# Save the result to a new Excel file or display it
result_df.to_excel('processed_isin_data_all_sheets.xlsx', index=False)
print(result_df)

In [14]:
import calendar
from datetime import datetime

def calculate_saturdays_in_year(year):
    total_saturdays = 0
    working_saturdays = 0

    for month in range(1, 13):  # Loop through all months
        _, num_days_in_month = calendar.monthrange(year, month)
        
        saturdays = []

        # Find all Saturdays in the month
        for day in range(1, num_days_in_month + 1):
            date = datetime(year, month, day)
            if date.weekday() == 5:  # Saturday is weekday 5
                saturdays.append(day)

        total_saturdays += len(saturdays)

        # Now apply the work/holiday rules
        holiday_saturdays = []

        # Rule 1: 2nd and 4th Saturdays are holidays
        if len(saturdays) >= 2:
            holiday_saturdays.append(saturdays[1])  # 2nd Saturday
        if len(saturdays) >= 4:
            holiday_saturdays.append(saturdays[3])  # 4th Saturday

        # Rule 2: If the last day of the month is a Monday, 4th Saturday becomes a working day
        last_day_of_month = datetime(year, month, num_days_in_month)
        if last_day_of_month.weekday() == 0:  # Monday is weekday 0
            if len(saturdays) >= 4 and saturdays[3] in holiday_saturdays:
                holiday_saturdays.remove(saturdays[3])  # Make 4th Saturday a working day

        # Rule 3: Always work on the 5th Saturday if present
        if len(saturdays) == 5 and saturdays[4] in holiday_saturdays:
            holiday_saturdays.remove(saturdays[4])  # Make 5th Saturday a working day

        # Calculate working Saturdays (not in holidays unless exceptions apply)
        working_saturdays += len(saturdays) - len(holiday_saturdays)

    return total_saturdays, working_saturdays

# Example usage for the year 2024
year = 2025
total_saturdays, working_saturdays = calculate_saturdays_in_year(year)

print(f"Total Saturdays in {year}: {total_saturdays}")
print(f"Saturdays you have to work in {year}: {working_saturdays}")
print(f"Saturdays you have holiday in {year}: {total_saturdays-working_saturdays}")

Total Saturdays in 2025: 52
Saturdays you have to work in 2025: 30
Saturdays you have holiday in 2025: 22


In [ ]:
import pandas as pd

# Function to calculate percentiles and cumulative sums for a given DataFrame
def calculate_percentiles(df, usd_col='USD_amount', counterparty_col='Counterparty', percentiles=[0.25, 0.5, 0.75, 1.0]):
    df = df.sort_values(by='Settlement_Date').reset_index(drop=True)  # Keep sorted by trade date
    df['Cumulative_USD'] = df[usd_col].cumsum()

    # Calculate percentiles based on USD_amount
    df['Percentile'] = pd.qcut(df[usd_col], q=len(percentiles), labels=[f'{int(p*100)}th' for p in percentiles])

    # Create result dictionary
    results = []

    for percentile in df['Percentile'].unique():
        percentile_df = df[df['Percentile'] == percentile]
        cumulative_usd = percentile_df['Cumulative_USD'].sum()
        trade_count = len(percentile_df)

        # Get top 5 counterparties for the percentile
        top_counterparties = (percentile_df.groupby(counterparty_col)[usd_col]
                              .sum().nlargest(5).index.tolist())

        results.append({
            'Percentile': percentile,
            'Cumulative_USD': cumulative_usd,
            'Trade_Count': trade_count,
            'Top_5_Counterparties': top_counterparties
        })

    return pd.DataFrame(results)

# Function to filter, calculate and save to Excel
def process_and_save_to_excel(df, file_name):
    # Filter the data into 3 categories
    equity_df = df[df['Instrument_Group'] == 'Equity']
    warr_debt_df = df[df['Instrument_Group'].isin(['WARR', 'Debt'])]
    curr_df = df[df['Instrument_Group'] == 'CURR']

    # Create an Excel writer
    with pd.ExcelWriter(file_name) as writer:
        # Process and save Equity
        equity_percentile_df = calculate_percentiles(equity_df)
        equity_percentile_df.to_excel(writer, sheet_name='Equity', index=False)

        # Process and save WARR and Debt
        warr_debt_percentile_df = calculate_percentiles(warr_debt_df)
        warr_debt_percentile_df.to_excel(writer, sheet_name='WARR_Debt', index=False)

        # Process and save CURR
        curr_percentile_df = calculate_percentiles(curr_df)
        curr_percentile_df.to_excel(writer, sheet_name='CURR', index=False)

# Example usage:
# df = pd.read_csv('your_trade_data.csv')  # Load your trade data
process_and_save_to_excel(df, 'trade_analysis.xlsx')


In [ ]:
import pandas as pd

def calculate_percentiles(df, usd_col='USD_amount', counterparty_col='Counterparty', settlement_col='Settlement_Date'):
    # Ensure data is sorted by settlement date
    df = df.sort_values(by=settlement_col).reset_index(drop=True)
    
    # Calculate cumulative USD amount
    df['Cumulative_USD'] = df[usd_col].cumsum()
    
    # Create 6 bins for percentiles using qcut
    df['Percentile'] = pd.qcut(df[usd_col], q=6, labels=[f'Bin {i+1}' for i in range(6)])
    
    results = []
    
    # Loop through each bin and gather required metrics
    for percentile in df['Percentile'].unique():
        percentile_df = df[df['Percentile'] == percentile]
        
        # Cumulative sum for the bin
        cumulative_usd = percentile_df['Cumulative_USD'].sum()
        
        # Count the number of trades in this percentile
        trade_count = len(percentile_df)

        # Top 5 counterparties by USD amount in this bin
        top_counterparties = (percentile_df.groupby(counterparty_col)[usd_col]
                              .sum().nlargest(5).index.tolist())
        
        results.append({
            'Percentile': percentile,
            'Cumulative_USD': cumulative_usd,
            'Trade_Count': trade_count,
            'Top_5_Counterparties': top_counterparties
        })
    
    return pd.DataFrame(results)

def filter_and_calculate(df, instrument_group, usd_col='USD_amount', counterparty_col='Counterparty', settlement_col='Settlement_Date'):
    # Filter by Instrument Group
    filtered_df = df[df['Instrument_Group'] == instrument_group]
    
    if len(filtered_df) > 0:
        # Calculate percentiles for the filtered data
        result_df = calculate_percentiles(filtered_df, usd_col, counterparty_col, settlement_col)
        return result_df
    else:
        return None

def process_and_save_to_excel(df, output_filename):
    with pd.ExcelWriter(output_filename) as writer:
        # Define the three filters for Instrument Group
        instrument_groups = ['Equity', 'WARR, debt', 'CURR']
        
        # Loop over each Instrument Group and calculate percentiles
        for group in instrument_groups:
            result_df = filter_and_calculate(df, group)
            if result_df is not None:
                # Save each group result in a separate sheet
                result_df.to_excel(writer, sheet_name=group, index=False)

# Example usage
df = pd.DataFrame({
    'Instrument_Group': ['Equity', 'Equity', 'WARR, debt', 'CURR', 'Equity', 'CURR'],
    'USD_amount': [100, 150, 200, 50, 300, 250],
    'Counterparty': ['A', 'B', 'C', 'D', 'E', 'F'],
    'Settlement_Date': pd.to_datetime(['2023-09-01', '2023-09-02', '2023-09-03', '2023-09-04', '2023-09-05', '2023-09-06'])
})

# Process the data and save results to Excel
process_and_save_to_excel(df, 'trade_percentiles.xlsx')
